In [108]:
from sklearn import preprocessing

def encode_text_index(df,name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

def encode_numeric_zscore(df,name,mean=None,sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name]-mean)/sd
    
def to_xy(df,target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)

    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    
    if target_type in (np.int64, np.int32):
        return df.as_matrix(result).astype(np.float32),df.as_matrix([target]).astype(np.int32)
    else:
        return df.as_matrix(result).astype(np.float32),df.as_matrix([target]).astype(np.float32)

In [112]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.contrib.learn as learn
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
tf.logging.set_verbosity(tf.logging.ERROR)
path = "/home/beshah/Desktop/Kaggle Project/Data"

filename_read = os.path.join(path,"train.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])


#x=df.loc[:,:]
#x.shape
#x,y = to_xy(df,'Outcome')
#x.shape
#y.shape

In [114]:

feature_columns =['A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16','A17','A18','A19']
df.drop('ID', axis=1)
x=df.loc[:,feature_columns]
y=df.Outcome
#Outcome = encode_text_index(df,'Outcome')
#num_classes = len(Outcome)
#model_dir = 'tmp/train' 
#x,y = to_xy(df,'Outcome')
#classifier = learn.DNNClassifier(
   # optimizer=tf.train.ProximalGradientDescentOptimizer( learning_rate=0.001,l1_regularization_strength=0.0,l2_regularization_strength=0.0),
    #model_dir= model_dir,
   # hidden_units=[40, 80, 20], n_classes=num_classes, feature_columns=feature_columns)
logreg = SGDClassifier()
    #hidden_units=[40, 80, 20], n_classes=2, feature_columns=feature_columns
                           
logreg.fit(x, y)
#pred = list(classifier.predict(x, as_iterable=True))


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [115]:
path = "/home/beshah/Desktop/Kaggle Project/Data"

filename_read = os.path.join(path,"test.csv")
dftest = pd.read_csv(filename_read,na_values=['NA','?'])

In [116]:
X_new = dftest.loc[:, feature_columns]
X_new.shape

(339, 18)

In [117]:
new_pred_class = logreg.predict(X_new)
pd.DataFrame({'ID':dftest.ID, 'Outcome':new_pred_class}).set_index('ID').to_csv('kagglesub14.csv')